# 0 Preparations
Before starting, ensure that you have cloned the repository to your Google Drive.
We will connect to this:

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
repository = 'evaluating_factuality_word_definitions'

%cd /content/drive/My Drive/{repository}

Mounted at /content/drive
/content/drive/My Drive/evaluating_factuality_word_definitions


Next, we install the packages and import the modules needed in this notebook:

In [2]:
%%capture
!pip install datasets~=2.18.0
!pip install einops~=0.8.0
!pip install rank_bm25~=0.2.2
!pip install openai~=1.35.10
!pip install git+https://github.com/tatuylonen/wiktextract.git

!python -m spacy download en_core_web_lg
!python -m spacy download de_core_news_lg

In [1]:
from collections import defaultdict
from typing import Dict, List

import json
import random
import os
import numpy as np
import torch
from datasets import load_dataset
from tqdm import tqdm

from config import PROJECT_DIR
from general_utils.reader import JSONLineReader, JSONReader
from general_utils.utils import (calc_bin_stats, print_classification_report,
                                 print_fever_classification_report, rank_docs)
from pipeline_module.claim_splitter import (DisSimSplitter, FactscoreSplitter,
                                            T5SplitRephraseSplitter)
from pipeline_module.evidence_fetcher import WikipediaEvidenceFetcher
from pipeline_module.evidence_selector import ModelEvidenceSelector
from pipeline_module.pipeline import FeverPipeline, Pipeline
from pipeline_module.sentence_connector import (ColonSentenceConnector,
                                                PhiSentenceConnector)
from pipeline_module.statement_verifier import ModelStatementVerifier
from pipeline_module.translator import OpusMTTranslator

# 1 Setup: Define Models and Datasets
Now we set the seed and define our models and datasets we want to evaluate:

In [2]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# to be deterministic
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [3]:
# Base Models
base_selection_model = 'Snowflake/snowflake-arctic-embed-m-long'
base_verification_model = 'MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7'

# Finetuned Models
finetuned_selection_model = 'lukasellinger/evidence-selection-model'
finetuned_verification_model = 'lukasellinger/claim-verification-model-top_last'

In [4]:
# Datasets with language information
datasets = {
    'german_dpr-claim_verification': {
        'dataset': load_dataset('lukasellinger/german_dpr-claim_verification', split='test'),
        'lang': 'de'
    },
    'german_wiktionary-claim_verification-mini': {
        'dataset': load_dataset('lukasellinger/german_wiktionary-claim_verification-mini', split='test'),
        'lang': 'de'
    },
    'squad-claim_verification': {
        'dataset': load_dataset('lukasellinger/squad-claim_verification', split='test'),
        'lang': 'en'
    },
    'shroom-claim_verification': {
        'dataset': load_dataset('lukasellinger/shroom-claim_verification', split='test'),
        'lang': 'en'
    }
    # outdated
    #'german-claim_verification': {
    #    'dataset': load_dataset('lukasellinger/german-claim_verification', split='test'),
    #    'lang': 'de'
    #},
}

optional_datasets = {
    'german_wiktionary-claim_verification-large': {
        'dataset': load_dataset('lukasellinger/german_wiktionary-claim_verification-large', split='test'),
        'lang': 'de'
    }
}

In [5]:
EVALUATION_DIR = PROJECT_DIR / 'data/evaluation'

## 1.1 Evaluation Util Functions
Next we define some helper functions

In [9]:
def evaluate_pipeline(pipeline: Pipeline, dataset, batch_size=4, output_file_name='',
                      only_intro=True, max_evidence_count: int = 3, top_k: int = 3, only_wikipedia: bool = False):
    outputs, report, not_in_wiki = pipeline.verify_test_dataset(dataset, batch_size,
                                                                output_file_name, only_intro, max_evidence_count, top_k, only_wikipedia)

    total_claim_count = sum(len(entry['atoms']) for entry in outputs if entry.get('atoms'))
    total_entries_with_atoms = sum(1 for entry in outputs if entry.get('atoms'))

    avg_claim_count = total_claim_count / total_entries_with_atoms if total_entries_with_atoms > 0 else 0

    print_classification_report(report, not_in_wiki, avg_claim_count)

    return outputs


def calc_claim_lengths_stats(outputs: List[Dict]):
    pr_labels, gt_labels, claim_lengths = [], [], []
    for output in outputs:
        pr_labels.append(1 if output['predicted'] == 'SUPPORTED' else 0)
        gt_labels.append(1 if output['label'] == 'SUPPORTED' else 0)
        claim_lengths.append(len(output['connected_claim'].split()))
    return calc_bin_stats(pr_labels, gt_labels, claim_lengths)


def calc_additional_stats(outputs: List[Dict], output_file_name=''):
    evid_line_number_dist = defaultdict(int)
    total_wikipedia = 0
    total_wiktionary = 0
    in_intro = 0

    for output in outputs:
        evidences = output.get('evidence', [])
        for evidence in evidences:
            if evidence.get('title').endswith('(wikipedia)'):
                if evidence.get('in_intro'):
                    in_intro += 1
                total_wikipedia += 1
            else:
                total_wiktionary += 1

            evid_line_number_dist[evidence.get('line_idx')] += 1
    total_evidences = total_wikipedia + total_wiktionary

    stats =  {
        'evid_line_number_dist': dict(sorted(evid_line_number_dist.items())),
        'claim_length_stats': calc_claim_lengths_stats(outputs),
        'avg_sent_0_selected': evid_line_number_dist.get(0) / sum(evid_line_number_dist.values()),
        'avg_in_intro': in_intro / total_wikipedia if total_wikipedia > 0 else 0,
        'in_intro': in_intro,
        'avg_wikipedia': total_wikipedia / total_evidences if total_evidences > 0 else 0,
        'total_wikipedia': total_wikipedia,
        'avg_wiktionary': total_wiktionary / total_evidences if total_evidences > 0 else 0,
        'total_wiktionary': total_wiktionary
    }
    if output_file_name:
        JSONReader().write(f'{output_file_name}.json', stats)

    return stats

## 1.2 Initialization of the pipeline modules
Here, we initialize the pipeline modules that will be used later.
These modules will be loaded onto your device when the first inference step is performed.
In this notebook, the translator and sentence connector are not directly utilized, but their results are already included in the datasets. We use their names to ensure the correct output is retrieved.

In [7]:
%%capture
# Translator
translator = OpusMTTranslator()

# Sentence Connectors
colon_sentence_connector = ColonSentenceConnector()
phi_sentence_connector = PhiSentenceConnector()

# Evidence Fetcher
offline_evid_fetcher = WikipediaEvidenceFetcher()
online_evid_fetcher = WikipediaEvidenceFetcher(offline=False)

In [8]:
pipeline_models = {
    'base': {
        'evid_selector': ModelEvidenceSelector(model_name=base_selection_model, min_similarity=0, evidence_selection='top'),
        'stm_verifier': ModelStatementVerifier(model_name=base_verification_model, premise_sent_order='keep')
    },
    'finetuned_wo_threshold': {
        'evid_selector': ModelEvidenceSelector(model_name=finetuned_selection_model, min_similarity=0, evidence_selection='top'),
        'stm_verifier': ModelStatementVerifier(model_name=finetuned_verification_model, premise_sent_order='top_last')
    },
    'finetuned_soft_threshold': { # best performing
        'evid_selector': ModelEvidenceSelector(model_name=finetuned_selection_model, min_similarity=0.5, evidence_selection='top'),
        'stm_verifier': ModelStatementVerifier(model_name=finetuned_verification_model, premise_sent_order='top_last')
    },
    'finetuned': {
        'evid_selector': ModelEvidenceSelector(model_name=finetuned_selection_model, min_similarity=0.585, evidence_selection='top'),
        'stm_verifier': ModelStatementVerifier(model_name=finetuned_verification_model, premise_sent_order='top_last')
    }
}

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

# 2 Evaluating Finetuned vs. Base Pipeline (Optimal Setup)

In this section, we assess the performance of the base and finetuned pipeline using the identified optimal setup.

#### Datasets and Models 
**Datasets:**
- [`lukasellinger/german_dpr-claim_verification`](https://huggingface.co/datasets/lukasellinger/german_dpr-claim_verification)
- [`lukasellinger/german_wiktionary-claim_verification-mini`](https://huggingface.co/datasets/lukasellinger/german_wiktionary-claim_verification-mini)
- [`lukasellinger/squad-claim_verification`](https://huggingface.co/datasets/lukasellinger/squad-claim_verification)
- [`lukasellinger/shroom-claim_verification`](https://huggingface.co/datasets/lukasellinger/shroom-claim_verification)
- [`lukasellinger/german_wiktionary-claim_verification-large`](https://huggingface.co/datasets/lukasellinger/german_wiktionary-claim_verification-large)

**Base Models:**
- Selection: [`Snowflake/snowflake-arctic-embed-m-long`](https://huggingface.co/Snowflake/snowflake-arctic-embed-m-long)
- Verification: [`MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7`](https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7)

**Finetuned Models:**
- Selection: [`lukasellinger/evidence-selection-model`](https://huggingface.co/lukasellinger/evidence-selection-model)
- Verification: [`lukasellinger/claim-verification-model-top_last`](https://huggingface.co/lukasellinger/claim-verification-model-top_last)


#### Evaluation Strategy

- **Configuration:**
  - Using `OpusMTTranslator`
  - Using `PhiSentenceConnector`
  - No claim splitting
  - Offline Evidence Fetcher for reproducibility (state of 08.07.2024)

## 2.1 Evaluation of Datasets
First we test our different pipeline versions on our test datasets.

In [ ]:
output_file_base_name = str(EVALUATION_DIR / "{dataset}/pipeline/{dataset}_{model}")
for dataset_name, config in datasets.items():
    dataset = config['dataset']
    lang = config['lang']
    for model_name, models in pipeline_models.items():
        print(f"Evaluating {dataset_name} with pipeline {model_name}...")
        pipeline = Pipeline(translator=translator,
                            sent_connector=phi_sentence_connector,
                            claim_splitter=None,
                            evid_fetcher=offline_evid_fetcher,
                            evid_selector=models.get('evid_selector'),
                            stm_verifier=models.get('stm_verifier'),
                            lang=lang)
        outputs = evaluate_pipeline(pipeline, dataset,
                                    output_file_name=output_file_base_name.format(
                                        dataset=dataset_name, model=model_name))
        additional_stats = calc_additional_stats(outputs, f'{output_file_base_name.format(dataset=dataset_name, model=model_name)}_additional_stats')

Evaluating german_dpr-claim_verification with pipeline base...


100%|██████████| 42/42 [01:13<00:00,  1.76s/it]


################################
Not in wikipedia: 29
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.6915    0.9420    0.7975        69
           1     0.9111    0.5857    0.7130        70

    accuracy                         0.7626       139
   macro avg     0.8013    0.7639    0.7553       139
weighted avg     0.8021    0.7626    0.7550       139

################################
Evaluating german_dpr-claim_verification with pipeline finetuned_wo_threshold...


100%|██████████| 42/42 [01:13<00:00,  1.75s/it]


################################
Not in wikipedia: 29
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.8451    0.8696    0.8571        69
           1     0.8676    0.8429    0.8551        70

    accuracy                         0.8561       139
   macro avg     0.8564    0.8562    0.8561       139
weighted avg     0.8564    0.8561    0.8561       139

################################
Evaluating german_dpr-claim_verification with pipeline finetuned_soft_threshold...


...

piss/English/noun: DEBUG: gloss may contain unhandled list items: 1999, Tin House #2 (→ISBN, Win McCormack, Rob Spillman, Elissa Schappell), page 170: at ['piss']


 58%|█████▊    | 82/141 [01:10<00:39,  1.48it/s]

demosophy/English/noun: DEBUG: unrecognized sense qualifier: folkloristics, sociology, rare at ['demosophy']


100%|██████████| 141/141 [02:13<00:00,  1.06it/s]

################################
Not in wikipedia: 21
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.6468    0.6259    0.6362       278
           1     0.6190    0.6402    0.6294       264

    accuracy                         0.6328       542
   macro avg     0.6329    0.6330    0.6328       542
weighted avg     0.6333    0.6328    0.6329       542

################################


## 2.2 Evaluation using only Wikipedia
Now we test our finetuned pipeline when using only Wikipedia Evidences (excluding Wiktionary).

In [14]:
output_file_base_name = str(EVALUATION_DIR / "{dataset}/pipeline/{dataset}_finetuned_soft_threshold_only_wikipedia")

for dataset_name, config in datasets.items():
    dataset = config['dataset']
    lang = config['lang']
    print(f"Evaluating {dataset_name}...")
    pipeline = Pipeline(translator=translator,
                        sent_connector=phi_sentence_connector,
                        claim_splitter=None,
                        evid_fetcher=offline_evid_fetcher,
                        evid_selector=pipeline_models['finetuned_soft_threshold']['evid_selector'],
                        stm_verifier=pipeline_models['finetuned_soft_threshold']['stm_verifier'],
                        lang=lang)
    outputs = evaluate_pipeline(pipeline, dataset, only_wikipedia=True,
                                output_file_name=output_file_base_name.format(dataset=dataset_name))

Evaluating german_dpr-claim_verification...


100%|██████████| 42/42 [08:20<00:00, 11.91s/it]


################################
Not in wikipedia: 32
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.8310    0.8806    0.8551        67
           1     0.8769    0.8261    0.8507        69

    accuracy                         0.8529       136
   macro avg     0.8540    0.8533    0.8529       136
weighted avg     0.8543    0.8529    0.8529       136

################################
Evaluating german_wiktionary-claim_verification-mini...


 12%|█▏        | 6/50 [00:20<02:36,  3.55s/it]

Imam/German/noun: DEBUG: unrecognized sense qualifier: Twelver Shiism at ['Imam']
imam/English/noun: DEBUG: unrecognized sense qualifier: Twelver Shi'ism at ['imam']


...

piss/English/noun: DEBUG: gloss may contain unhandled list items: 1999, Tin House #2 (→ISBN, Win McCormack, Rob Spillman, Elissa Schappell), page 170: at ['piss']


 58%|█████▊    | 82/141 [01:26<01:00,  1.03s/it]

demosophy/English/noun: DEBUG: unrecognized sense qualifier: folkloristics, sociology, rare at ['demosophy']


100%|██████████| 141/141 [02:40<00:00,  1.14s/it]


################################
Not in wikipedia: 416
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.6842    0.6420    0.6624        81
           1     0.5915    0.6364    0.6131        66

    accuracy                         0.6395       147
   macro avg     0.6379    0.6392    0.6378       147
weighted avg     0.6426    0.6395    0.6403       147

################################


## 2.3 Evaluation using ColonSentenceConnector
In our best performance we used PhiSentenceConnector. 
However this models takes up over 90% of our total parameters. 
Therefore, we test the perfomance when adding Word and Definition with: `{word}: {definition}`

In [ ]:
output_file_base_name = str(EVALUATION_DIR / "{dataset}/pipeline/{dataset}_{model}_colon_connector")
for dataset_name, config in datasets.items():
    dataset = config['dataset']
    lang = config['lang']
    for model_name, models in pipeline_models.items():
        print(f"Evaluating {dataset_name} with pipeline {model_name}...")
        pipeline = Pipeline(translator=translator,
                            sent_connector=colon_sentence_connector,
                            claim_splitter=None,
                            evid_fetcher=offline_evid_fetcher,
                            evid_selector=models.get('evid_selector'),
                            stm_verifier=models.get('stm_verifier'),
                            lang=lang)
        outputs = evaluate_pipeline(pipeline, dataset,
                                    output_file_name=output_file_base_name.format(
                                        dataset=dataset_name, model=model_name))
        additional_stats = calc_additional_stats(outputs, f'{output_file_base_name.format(dataset=dataset_name, model=model_name)}_additional_stats')

## 2.4 Evaluation of Wiktionary-Large Dataset (optional)
Next we test our finetuned pipeline on the Wiktionary-Large Dataset containing 10k entries.

In [ ]:
output_file_base_name = str(EVALUATION_DIR / "{dataset}/pipeline/{dataset}_finetuned_soft_threshold")
dataset_name = 'german_wiktionary-claim_verification-large'
dataset = optional_datasets[dataset_name]['dataset']
lang = optional_datasets[dataset_name]['lang']
print(f"Evaluating {dataset_name}...")
pipeline = Pipeline(translator=translator,
                    sent_connector=phi_sentence_connector,
                    claim_splitter=None,
                    evid_fetcher=offline_evid_fetcher,
                    evid_selector=pipeline_models['finetuned_soft_threshold']['evid_selector'],
                    stm_verifier=pipeline_models['finetuned_soft_threshold']['stm_verifier'],
                    lang=lang)
outputs = evaluate_pipeline(pipeline, dataset,
                            output_file_name=output_file_base_name.format(dataset=dataset_name))

Evaluating german_wiktionary-claim_verification-large...


  0%|          | 6/2500 [00:06<46:00,  1.11s/it]

Imam/German/noun: DEBUG: unrecognized sense qualifier: Twelver Shiism at ['Imam']
imam/English/noun: DEBUG: unrecognized sense qualifier: Twelver Shi'ism at ['imam']


  1%|          | 14/2500 [00:15<42:09,  1.02s/it]

unterbinden/German/verb: DEBUG: unrecognized sense qualifier: transitive or dative reflexive, dated or regional at ['unterbinden']


...

100%/English/noun: DEBUG: unrecognized sense qualifier: speedrunning at ['100%']


 99%|█████████▉| 2487/2500 [59:21<00:12,  1.03it/s]

split/English/noun: DEBUG: unrecognized sense qualifier: athletics, speedrunning at ['split']


100%|██████████| 2500/2500 [59:41<00:00,  1.43s/it]


################################
Not in wikipedia: 2170
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.6854    0.8720    0.7675      3938
           1     0.8213    0.5951    0.6901      3892

    accuracy                         0.7344      7830
   macro avg     0.7534    0.7335    0.7288      7830
weighted avg     0.7530    0.7344    0.7291      7830

################################


# 3 Evaluating Finetuned Pipeline Online (Best Setup)
In the previous section, we evaluated our finetuned pipeline using the offline evidence fetcher. We can also connect our pipeline to the Wikipedia API to retrieve the most current knowledge available.

Let's check on that:

In [ ]:
output_file_base_name = str(EVALUATION_DIR / "{dataset}/pipeline/{dataset}_finetuned_soft_threshold_online")

for dataset_name, config in datasets.items():
    dataset = config['dataset']
    lang = config['lang']
    print(f"Evaluating {dataset_name}...")
    pipeline = Pipeline(translator=translator,
                        sent_connector=phi_sentence_connector,
                        claim_splitter=None,
                        evid_fetcher=online_evid_fetcher,
                        evid_selector=pipeline_models['finetuned_soft_threshold']['evid_selector'],
                        stm_verifier=pipeline_models['finetuned_soft_threshold']['stm_verifier'],
                        lang=lang)
    evaluate_pipeline(pipeline, dataset,
                      output_file_name=output_file_base_name.format(dataset=dataset_name))

Evaluating german_dpr-claim_verification...


100%|██████████| 42/42 [02:34<00:00,  3.67s/it]


################################
Not in wikipedia: 29
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.8219    0.8696    0.8451        69
           1     0.8636    0.8143    0.8382        70

    accuracy                         0.8417       139
   macro avg     0.8428    0.8419    0.8417       139
weighted avg     0.8429    0.8417    0.8416       139

################################
Evaluating german_wiktionary-claim_verification-mini...


 12%|█▏        | 6/50 [00:28<03:31,  4.80s/it]

Imam/German/noun: DEBUG: unrecognized sense qualifier: Twelver Shiism at ['Imam']
imam/English/noun: DEBUG: unrecognized sense qualifier: Twelver Shi'ism at ['imam']
imam/English/noun: DEBUG: unrecognized sense qualifier: Twelver Shi'ism at ['imam']


...

piss/English/noun: DEBUG: gloss may contain unhandled list items: 1999, Tin House #2 (→ISBN, Win McCormack, Rob Spillman, Elissa Schappell), page 170: at ['piss']


 58%|█████▊    | 82/141 [05:15<04:13,  4.30s/it]

demosophy/English/noun: DEBUG: unrecognized sense qualifier: folkloristics, sociology, rare at ['demosophy']


100%|██████████| 141/141 [08:55<00:00,  3.80s/it]


################################
Not in wikipedia: 21
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.7042    0.5396    0.6110       278
           1     0.6109    0.7614    0.6779       264

    accuracy                         0.6476       542
   macro avg     0.6576    0.6505    0.6445       542
weighted avg     0.6588    0.6476    0.6436       542

################################


# 4 Evaluation of Different Claim Splitters

A sentence can be split into multiple facts, where the combination of these facts represents the entire sentence.

For this evaluation, we test four different splitters:

- **`DisSimSplitter`**: Based on [DiscourseSimplification](https://github.com/Lambda-3/DiscourseSimplification)
- **`T5SplitRephraseSplitter`**: Based on [T5 Split and Rephrase](https://huggingface.co/unikei/t5-base-split-and-rephrase)
- **`FactscoreSplitter`**: Based on [FActScore](https://github.com/shmsw25/FActScore)
- **`None`**: No splitting

In [ ]:
%%capture
claim_splitters = {
    'DisSimSplitter': DisSimSplitter(),
    'T5SplitRephraseSplitter': T5SplitRephraseSplitter(),
    'FactscoreSplitter': FactscoreSplitter()
}

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In this setting, the splits are not calculated. Instead, they are reused as they have already been precomputed and are present in the datasets.

In [ ]:
output_file_base_name = str(EVALUATION_DIR / "{dataset}/pipeline/{dataset}_finetuned_soft_threshold_{splitter}")

for dataset_name, config in datasets.items():
    dataset = config['dataset']
    lang = config['lang']
    for name, splitter in claim_splitters.items():
        print(f"Evaluating {dataset_name} with claim splitter {name}...")
        pipeline = Pipeline(translator=translator,
                            sent_connector=phi_sentence_connector,
                            claim_splitter=splitter,
                            evid_fetcher=offline_evid_fetcher,
                            evid_selector=pipeline_models['finetuned_soft_threshold']['evid_selector'],
                            stm_verifier=pipeline_models['finetuned_soft_threshold']['stm_verifier'],
                            lang=lang)
        evaluate_pipeline(pipeline, dataset,
                          output_file_name=output_file_base_name.format(dataset=dataset_name,
                                                                        splitter=name))

Evaluating german_dpr-claim_verification with claim splitter DisSimSplitter...


100%|██████████| 42/42 [00:50<00:00,  1.21s/it]


################################
Not in wikipedia: 29
Avg claim count: 1.870503597122302
              precision    recall  f1-score   support

           0     0.7283    0.9710    0.8323        69
           1     0.9574    0.6429    0.7692        70

    accuracy                         0.8058       139
   macro avg     0.8429    0.8069    0.8008       139
weighted avg     0.8437    0.8058    0.8005       139

################################
Evaluating german_dpr-claim_verification with claim splitter T5SplitRephraseSplitter...


100%|██████████| 42/42 [00:49<00:00,  1.17s/it]


################################
Not in wikipedia: 29
Avg claim count: 1.474820143884892
              precision    recall  f1-score   support

           0     0.7158    0.9855    0.8293        69
           1     0.9773    0.6143    0.7544        70

    accuracy                         0.7986       139
   macro avg     0.8465    0.7999    0.7918       139
weighted avg     0.8475    0.7986    0.7916       139

################################
Evaluating german_dpr-claim_verification with claim splitter FactscoreSplitter...


...

piss/English/noun: DEBUG: gloss may contain unhandled list items: 1999, Tin House #2 (→ISBN, Win McCormack, Rob Spillman, Elissa Schappell), page 170: at ['piss']


 58%|█████▊    | 82/141 [01:01<00:46,  1.27it/s]

demosophy/English/noun: DEBUG: unrecognized sense qualifier: folkloristics, sociology, rare at ['demosophy']


100%|██████████| 141/141 [01:46<00:00,  1.33it/s]

################################
Not in wikipedia: 21
Avg claim count: 2.726937269372694
              precision    recall  f1-score   support

           0     0.6667    0.7554    0.7083       278
           1     0.7004    0.6023    0.6477       264

    accuracy                         0.6808       542
   macro avg     0.6836    0.6788    0.6780       542
weighted avg     0.6831    0.6808    0.6787       542

################################


# 5 Whole Wiki Page Run (Best Setup)

In our previous setting, we limited the text fetched to the intro sections of Wikipedia pages. However, we could extend this to use the entire Wikipedia page, which would provide more information but would significantly increase the processing time.

Let us try it out:


In [ ]:
output_file_base_name = str(EVALUATION_DIR / "{dataset}/pipeline/{dataset}_finetuned_soft_threshold_whole_page")

for dataset_name, config in datasets.items():
    print(f"Evaluating {dataset_name}...")
    dataset = config['dataset']
    lang = config['lang']

    pipeline = Pipeline(translator=translator,
                        sent_connector=phi_sentence_connector,
                        claim_splitter=None,
                        evid_fetcher=offline_evid_fetcher,
                        evid_selector=pipeline_models['finetuned_soft_threshold']['evid_selector'],
                        stm_verifier=pipeline_models['finetuned_soft_threshold']['stm_verifier'],
                        lang=lang)
    outputs = evaluate_pipeline(pipeline, dataset,
                                output_file_name=output_file_base_name.format(dataset=dataset_name),
                                only_intro=False)
    additional_stats = calc_additional_stats(outputs, f'{output_file_base_name.format(dataset=dataset_name)}_additional_stats')

Evaluating german_dpr-claim_verification...


  0%|          | 0/42 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

configuration_hf_nomic_bert.py:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Snowflake/snowflake-arctic-embed-m-long:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

piss/English/noun: DEBUG: gloss may contain unhandled list items: 1999, Tin House #2 (→ISBN, Win McCormack, Rob Spillman, Elissa Schappell), page 170: at ['piss']


 58%|█████▊    | 82/141 [03:05<02:00,  2.04s/it]

demosophy/English/noun: DEBUG: unrecognized sense qualifier: folkloristics, sociology, rare at ['demosophy']


100%|██████████| 141/141 [05:37<00:00,  2.39s/it]


################################
Not in wikipedia: 21
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.6875    0.5540    0.6135       278
           1     0.6101    0.7348    0.6667       264

    accuracy                         0.6421       542
   macro avg     0.6488    0.6444    0.6401       542
weighted avg     0.6498    0.6421    0.6394       542

################################


# 6 Evaluate FEVER Score

Here, we test our fine-tuning process with the FEVER dataset using three evaluation metrics: FeverScore, Gold Label, and Gold Evidence. 

1. **FeverScore**: This metric evaluates strict label accuracy. It considers an entry to be correctly classified only if at least one relevant piece of evidence is present in the selected evidence sentences. This ensures that a correct classification without supporting evidence is not counted as correctly classified. Entries labeled *Not Enough Info* are considered correctly classified without evidence, as no annotated evidence is available. Similarly, claims classified as unsupported due to topic modeling are considered correct as no evidence is selected.

2. **Gold Label**: In this metric, we provide the gold labels to the FeverScore. This allows us to measure the pipeline's ability to retrieve correct evidence without considering downstream statement classification.

3. **Gold Evidence**: We provide the gold evidence directly to the statement classification component for this metric. This allows us to assess its accuracy independently of the evidence retrieval step.

Important: Gold Label + Gold Evidence does not necessarly lead a Gold Label socre of 1 as there exist entries which have evidence groups that are larger than 3 and thus cannot be fully captured as we do only select 3 sentences.

In [ ]:
def evaluate_fever_pipeline(pipeline: FeverPipeline, dataset, batch_size=4, output_file_name='', gold_evidence=False):
    outputs, report, fever_report = pipeline.verify_test_dataset(dataset, batch_size, output_file_name, gold_evidence=gold_evidence)
    print_fever_classification_report(report, fever_report)
    return outputs

## 6.1 Including Not Enough Info entries

We report the results on the FEVER dataset when including Not Enough Info entries.
In training we excluded them, as this improved performance.

In [ ]:
output_file_base_name = str(EVALUATION_DIR / "fever/fever_{name}")
dataset = load_dataset("lukasellinger/filtered_fever-claim_verification", split='test')
# dataset = dataset.filter(lambda x: x['label'] != 'NOT_ENOUGH_INFO')

Generating train split:   0%|          | 0/38466 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/2936 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2469 [00:00<?, ? examples/s]

In [ ]:
for name, models in pipeline_models.items():
    print(f"Evaluating with {name}...")
    pipeline = FeverPipeline(claim_splitter=None,
                             evid_selector=models.get('evid_selector'),
                             stm_verifier=models.get('stm_verifier'))
    evaluate_fever_pipeline(pipeline, dataset, output_file_name=output_file_base_name.format(name=name), gold_evidence=False)
    print(f'Evaluating with {name} and gold evidence....')
    evaluate_fever_pipeline(pipeline, dataset, output_file_name=output_file_base_name.format(name=name), gold_evidence=True)

Evaluating with base...


  0%|          | 0/618 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

configuration_hf_nomic_bert.py:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Snowflake/snowflake-arctic-embed-m-long:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

################################
FeverScore: 0.9459940652818991
Gold Label: 0.998813056379822
              precision    recall  f1-score   support

           0     0.9689    0.9279    0.9480       874
           1     0.9257    0.9679    0.9464       811

    accuracy                         0.9472      1685
   macro avg     0.9473    0.9479    0.9472      1685
weighted avg     0.9481    0.9472    0.9472      1685

################################
Evaluating with finetuned...


100%|██████████| 618/618 [05:38<00:00,  1.83it/s]


################################
FeverScore: 0.8683677602268125
Gold Label: 0.97893884163629
              precision    recall  f1-score   support

           0     0.9661    0.8601    0.9100      1658
           1     0.7664    0.9383    0.8437       811

    accuracy                         0.8858      2469
   macro avg     0.8662    0.8992    0.8768      2469
weighted avg     0.9005    0.8858    0.8882      2469

################################
Evaluating with finetuned and gold evidence....


100%|██████████| 618/618 [01:40<00:00,  6.14it/s]


################################
FeverScore: 0.9459940652818991
Gold Label: 0.998813056379822
              precision    recall  f1-score   support

           0     0.9689    0.9279    0.9480       874
           1     0.9257    0.9679    0.9464       811

    accuracy                         0.9472      1685
   macro avg     0.9473    0.9479    0.9472      1685
weighted avg     0.9481    0.9472    0.9472      1685

################################


## 6.2 Excluding Not Enough Info Entries

Next, we report the results on the FEVER dataset when we exlcude the Not Enough Info entries.
In our training we also excluded them.

In [ ]:
output_file_base_name = str(EVALUATION_DIR / "fever/fever_filtered_{name}")
dataset = load_dataset("lukasellinger/filtered_fever-claim_verification", split='test')
dataset = dataset.filter(lambda x: x['label'] != 'NOT_ENOUGH_INFO')

In [ ]:
for name, models in pipeline_models.items():
    print(f"Evaluating with {name}...")
    pipeline = FeverPipeline(claim_splitter=None,
                             evid_selector=models.get('evid_selector'),
                             stm_verifier=models.get('stm_verifier'))
    evaluate_fever_pipeline(pipeline, dataset, output_file_name=output_file_base_name.format(name=name), gold_evidence=False)
    print(f'Evaluating with {name} and gold evidence....')
    evaluate_fever_pipeline(pipeline, dataset, output_file_name=output_file_base_name.format(name=name), gold_evidence=True)

Evaluating with base...


  0%|          | 0/422 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

...

################################
FeverScore: 0.9459940652818991
Gold Label: 0.998813056379822
              precision    recall  f1-score   support

           0     0.9689    0.9279    0.9480       874
           1     0.9257    0.9679    0.9464       811

    accuracy                         0.9472      1685
   macro avg     0.9473    0.9479    0.9472      1685
weighted avg     0.9481    0.9472    0.9472      1685

################################
Evaluating with finetuned...


100%|██████████| 422/422 [03:48<00:00,  1.84it/s]


################################
FeverScore: 0.9068249258160237
Gold Label: 0.9691394658753709
              precision    recall  f1-score   support

           0     0.9419    0.9268    0.9343       874
           1     0.9224    0.9383    0.9303       811

    accuracy                         0.9323      1685
   macro avg     0.9321    0.9326    0.9323      1685
weighted avg     0.9325    0.9323    0.9324      1685

################################
Evaluating with finetuned and gold evidence....


100%|██████████| 422/422 [01:29<00:00,  4.71it/s]


################################
FeverScore: 0.9459940652818991
Gold Label: 0.998813056379822
              precision    recall  f1-score   support

           0     0.9689    0.9279    0.9480       874
           1     0.9257    0.9679    0.9464       811

    accuracy                         0.9472      1685
   macro avg     0.9473    0.9479    0.9472      1685
weighted avg     0.9481    0.9472    0.9472      1685

################################


## 6.1 MMR vs Top Evidence Selection
We check on our development set, which evidence selection method works best. We test:
- Maximal Marginal Relevance (MMR)
- Maximum Similarity

In [ ]:
output_file_base_name = str(EVALUATION_DIR / "fever/fever_{name}_selection")
dataset = load_dataset("lukasellinger/filtered_fever-claim_verification", split='dev')
dataset = dataset.filter(lambda x: x['label'] != 'NOT_ENOUGH_INFO')

Generating train split:   0%|          | 0/38466 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/2936 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2469 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2936 [00:00<?, ? examples/s]

In [ ]:
pipeline = FeverPipeline(claim_splitter=None,
                         evid_selector=ModelEvidenceSelector(model_name=finetuned_selection_model, min_similarity=0, evidence_selection='mmr'),
                         stm_verifier= ModelStatementVerifier(model_name=base_verification_model, premise_sent_order='top_last'))

tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
evidence_selections = ['mmr', 'top']
for evidence_selection in evidence_selections:
    print(f"Evaluating with {evidence_selection}...")
    pipeline.evid_selector.set_evidence_selection(evidence_selection)
    evaluate_fever_pipeline(pipeline, dataset, output_file_name=output_file_base_name.format(name=evidence_selection))

Evaluating with mmr...


  0%|          | 0/495 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

configuration_hf_nomic_bert.py:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Snowflake/snowflake-arctic-embed-m-long:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

100%|██████████| 495/495 [04:19<00:00,  1.91it/s]


################################
FeverScore: 0.9277047522750252
Gold Label: 0.9691607684529828
              precision    recall  f1-score   support

           0     0.9426    0.9640    0.9532      1056
           1     0.9577    0.9328    0.9451       922

    accuracy                         0.9494      1978
   macro avg     0.9501    0.9484    0.9491      1978
weighted avg     0.9496    0.9494    0.9494      1978

################################
Evaluating with top...


100%|██████████| 495/495 [04:07<00:00,  2.00it/s]

################################
FeverScore: 0.9322548028311426
Gold Label: 0.9732052578361982
              precision    recall  f1-score   support

           0     0.9470    0.9650    0.9559      1056
           1     0.9590    0.9382    0.9485       922

    accuracy                         0.9525      1978
   macro avg     0.9530    0.9516    0.9522      1978
weighted avg     0.9526    0.9525    0.9524      1978

################################


## 6.2 Find Sentence Ordering
Next, we check on our development set, which sentence ordering in the premise of our statement classifier works best. We test:
- Original (keep)
- Reversed (reverse) 
- Most promising at End (top_last)

In [ ]:
dataset = load_dataset("lukasellinger/filtered_fever-claim_verification", split='dev')
#dataset = dataset.filter(lambda x: x['label'] != 'NOT_ENOUGH_INFO')
output_file_base_name = str(EVALUATION_DIR / "fever/fever_{name}_order")

In [ ]:
model_name = 'lukasellinger/claim-verification-model'
pipeline_models = {
    'keep': {
        'evid_selector': ModelEvidenceSelector(model_name=finetuned_selection_model, min_similarity=0, evidence_selection='top'),
        'stm_verifier': ModelStatementVerifier(model_name=f'{model_name}-keep', premise_sent_order='keep')
    },
    'reverse': {
        'evid_selector': ModelEvidenceSelector(model_name=finetuned_selection_model, min_similarity=0, evidence_selection='top'),
        'stm_verifier': ModelStatementVerifier(model_name=f'{model_name}-reverse', premise_sent_order='reverse')
    },
    'top_last': {
        'evid_selector': ModelEvidenceSelector(model_name=finetuned_selection_model, min_similarity=0, evidence_selection='top'),
        'stm_verifier': ModelStatementVerifier(model_name=f'{model_name}-top_last', premise_sent_order='top_last')
    }
}

In [ ]:
for name, models in pipeline_models.items():
    print(f"Evaluating with sentence order: {name}...")
    pipeline = FeverPipeline(claim_splitter=None,
                             evid_selector=models.get('evid_selector'),
                             stm_verifier=models.get('stm_verifier'))
    evaluate_fever_pipeline(pipeline, dataset, output_file_name=output_file_base_name.format(name=name))

Evaluating with sentence order: keep...


100%|██████████| 734/734 [06:10<00:00,  1.98it/s]


################################
FeverScore: 0.8746594005449592
Gold Label: 0.9819482288828338
              precision    recall  f1-score   support

           0     0.9817    0.8540    0.9134      2014
           1     0.7517    0.9653    0.8452       922

    accuracy                         0.8890      2936
   macro avg     0.8667    0.9097    0.8793      2936
weighted avg     0.9095    0.8890    0.8920      2936

################################
Evaluating with sentence order: reverse...


100%|██████████| 734/734 [06:14<00:00,  1.96it/s]


################################
FeverScore: 0.8763623978201635
Gold Label: 0.9819482288828338
              precision    recall  f1-score   support

           0     0.9780    0.8600    0.9152      2014
           1     0.7579    0.9577    0.8462       922

    accuracy                         0.8907      2936
   macro avg     0.8680    0.9088    0.8807      2936
weighted avg     0.9089    0.8907    0.8935      2936

################################
Evaluating with sentence order: top_last...


100%|██████████| 734/734 [06:08<00:00,  1.99it/s]


################################
FeverScore: 0.877724795640327
Gold Label: 0.9819482288828338
              precision    recall  f1-score   support

           0     0.9775    0.8630    0.9167      2014
           1     0.7617    0.9566    0.8481       922

    accuracy                         0.8924      2936
   macro avg     0.8696    0.9098    0.8824      2936
weighted avg     0.9097    0.8924    0.8951      2936

################################


## 6.3 Find Threshold
Finally, we test on our development set, which sentence similarity threshold should be used in our Evidence Selection (Topic Modeling)

In [ ]:
dataset = load_dataset("lukasellinger/filtered_fever-claim_verification", split='dev')
#dataset = dataset.filter(lambda x: x['label'] != 'NOT_ENOUGH_INFO')
output_file_base_name = str(EVALUATION_DIR / "fever/fever_{threshold}_threshold")

In [ ]:
model_name = 'lukasellinger/claim-verification-model-top_last'
pipeline = FeverPipeline(claim_splitter=None,
                         evid_selector= ModelEvidenceSelector(model_name=finetuned_selection_model, min_similarity=0, evidence_selection='top'),
                         stm_verifier= ModelStatementVerifier(model_name=model_name, premise_sent_order='top_last'))

In [ ]:
start = 480
stop = 536
step = 5  # corresponds to 0.005 increments

for threshold_int in range(start, stop, step):
    print(f"Evaluating with threshold: {threshold_int}...")
    threshold = threshold_int / 1000.0  # Convert back to the floating-point value
    pipeline.evid_selector.set_min_similarity(threshold)
    evaluate_fever_pipeline(pipeline, dataset, output_file_name=output_file_base_name.format(threshold=threshold_int))

Evaluating with threshold: 480...


100%|██████████| 734/734 [06:06<00:00,  2.00it/s]


################################
FeverScore: 0.877724795640327
Gold Label: 0.9816076294277929
              precision    recall  f1-score   support

           0     0.9759    0.8649    0.9171      2014
           1     0.7637    0.9534    0.8480       922

    accuracy                         0.8927      2936
   macro avg     0.8698    0.9092    0.8826      2936
weighted avg     0.9093    0.8927    0.8954      2936

################################
Evaluating with threshold: 485...


100%|██████████| 734/734 [06:01<00:00,  2.03it/s]


################################
FeverScore: 0.877724795640327
Gold Label: 0.9816076294277929
              precision    recall  f1-score   support

           0     0.9754    0.8654    0.9171      2014
           1     0.7641    0.9523    0.8479       922

    accuracy                         0.8927      2936
   macro avg     0.8698    0.9089    0.8825      2936
weighted avg     0.9090    0.8927    0.8954      2936

################################
Evaluating with threshold: 490...


...

################################
FeverScore: 0.8780653950953679
Gold Label: 0.9812670299727521
              precision    recall  f1-score   support

           0     0.9743    0.8669    0.9175      2014
           1     0.7657    0.9501    0.8480       922

    accuracy                         0.8931      2936
   macro avg     0.8700    0.9085    0.8828      2936
weighted avg     0.9088    0.8931    0.8957      2936

################################
Evaluating with threshold: 530...


100%|██████████| 734/734 [06:02<00:00,  2.02it/s]


################################
FeverScore: 0.8784059945504087
Gold Label: 0.9812670299727521
              precision    recall  f1-score   support

           0     0.9738    0.8679    0.9178      2014
           1     0.7669    0.9490    0.8483       922

    accuracy                         0.8934      2936
   macro avg     0.8703    0.9085    0.8831      2936
weighted avg     0.9088    0.8934    0.8960      2936

################################
Evaluating with threshold: 535...


100%|██████████| 734/734 [06:02<00:00,  2.03it/s]


################################
FeverScore: 0.8780653950953679
Gold Label: 0.9812670299727521
              precision    recall  f1-score   support

           0     0.9733    0.8679    0.9176      2014
           1     0.7667    0.9479    0.8477       922

    accuracy                         0.8931      2936
   macro avg     0.8700    0.9079    0.8827      2936
weighted avg     0.9084    0.8931    0.8956      2936

################################


In [ ]:
start = 540
stop = 601
step = 5  # corresponds to 0.005 increments

for threshold_int in range(start, stop, step):
    print(f"Evaluating with threshold: {threshold_int}...")
    threshold = threshold_int / 1000.0  # Convert back to the floating-point value
    pipeline.evid_selector.set_min_similarity(threshold)
    evaluate_fever_pipeline(pipeline, dataset, output_file_name=output_file_base_name.format(threshold=threshold_int))

Evaluating with threshold: 540...


100%|██████████| 734/734 [05:59<00:00,  2.04it/s]


################################
FeverScore: 0.8787465940054496
Gold Label: 0.9816076294277929
              precision    recall  f1-score   support

           0     0.9722    0.8694    0.9180      2014
           1     0.7683    0.9458    0.8478       922

    accuracy                         0.8934      2936
   macro avg     0.8703    0.9076    0.8829      2936
weighted avg     0.9082    0.8934    0.8959      2936

################################
Evaluating with threshold: 545...


100%|██████████| 734/734 [05:56<00:00,  2.06it/s]


################################
FeverScore: 0.8787465940054496
Gold Label: 0.9822888283378747
              precision    recall  f1-score   support

           0     0.9712    0.8699    0.9178      2014
           1     0.7686    0.9436    0.8471       922

    accuracy                         0.8931      2936
   macro avg     0.8699    0.9068    0.8824      2936
weighted avg     0.9075    0.8931    0.8956      2936

################################
Evaluating with threshold: 550...


...

################################
FeverScore: 0.8790871934604905
Gold Label: 0.9819482288828338
              precision    recall  f1-score   support

           0     0.9625    0.8793    0.9190      2014
           1     0.7783    0.9252    0.8454       922

    accuracy                         0.8937      2936
   macro avg     0.8704    0.9023    0.8822      2936
weighted avg     0.9047    0.8937    0.8959      2936

################################
Evaluating with threshold: 595...


100%|██████████| 734/734 [05:48<00:00,  2.11it/s]


################################
FeverScore: 0.8804495912806539
Gold Label: 0.9822888283378747
              precision    recall  f1-score   support

           0     0.9621    0.8813    0.9199      2014
           1     0.7809    0.9241    0.8465       922

    accuracy                         0.8948      2936
   macro avg     0.8715    0.9027    0.8832      2936
weighted avg     0.9052    0.8948    0.8969      2936

################################
Evaluating with threshold: 600...


100%|██████████| 734/734 [05:46<00:00,  2.12it/s]


################################
FeverScore: 0.8780653950953679
Gold Label: 0.9812670299727521
              precision    recall  f1-score   support

           0     0.9580    0.8833    0.9191      2014
           1     0.7822    0.9154    0.8436       922

    accuracy                         0.8934      2936
   macro avg     0.8701    0.8994    0.8814      2936
weighted avg     0.9028    0.8934    0.8954      2936

################################


# 7 Check on incorrect predicted

Similar to the approach used in FactScore, we sample 30 incorrect predictions from our test sets, excluding those cases where no evidence is available on Wikipedia. These sampled examples are saved to a separate file for detailed manual evaluation. The analysis focuses on identifying the following types of issues:
- Predicted as Supported
- Annotation
- No Evidence in Wiki
- No Evidence in Selection
- Pre-processing (e.g translation error)

In [ ]:
def sample_incorrect_predictions(dataset_names: List[str], file_pattern: str, output_file: str,
                                 sample_size: int = 30, seed: int = 42) -> None:
    """
    Samples incorrect predictions from test sets and saves them to a file for manual assessment.

    Args:
        dataset_names (List[str]): List of dataset names to process.
        file_pattern (str): Pattern to match file names for loading predictions.
        output_file (str): Path to the output file for saving the sampled incorrect predictions.
        sample_size (int): Number of incorrect predictions to sample.
        seed (int): Random seed for reproducibility.
    """
    random.seed(seed)
    all_outputs = []

    for dataset_name in dataset_names:
        file_path = EVALUATION_DIR / file_pattern.format(dataset_name=dataset_name)
        outputs = JSONLineReader().read(file_path)
        all_outputs.extend(outputs)
    false_predicted_outputs = [output for output in all_outputs if
                               output['label'] != output['predicted'] and output['predicted'] != -1]

    sample_size = min(sample_size, len(false_predicted_outputs))
    sampled_outputs = random.sample(false_predicted_outputs, sample_size)

    # Print sampled outputs for review
    print(f"Sampled {len(sampled_outputs)} incorrect predictions:")
    for sample in sampled_outputs:
        print(json.dumps(sample, indent=4))
        print('############################')
    JSONLineReader().write(output_file, sampled_outputs)

In [ ]:
# Sample incorrect predictions from the fine-tuned pipeline without Claim Splitting
sample_incorrect_predictions(
    dataset_names=list(datasets.keys()),
    file_pattern="{dataset_name}/pipeline/{dataset_name}_finetuned_soft_threshold.jsonl",
    output_file=str(EVALUATION_DIR / "incorrect_pred_samples.jsonl")
)

# Sample incorrect predictions from models with DisSimSplitter
sample_incorrect_predictions(
    dataset_names=list(datasets.keys()),
    file_pattern="{dataset_name}/pipeline/{dataset_name}_finetuned_soft_threshold_DisSimSplitter.jsonl",
    output_file=str(EVALUATION_DIR / "incorrect_pred_samples_DisSimSplitter.jsonl")
)

Sampled 30 incorrect predictions:
{
    "id": 6,
    "word": "dharmas",
    "claim": "phenomena",
    "connected_claim": "Dharmas signifies phenomena.",
    "label": "SUPPORTED",
    "predicted": "NOT_SUPPORTED",
    "factuality": 0.0,
    "atoms": [
        {
            "atom": "Dharmas signifies phenomena.",
            "predicted": "NOT_SUPPORTED"
        }
    ],
    "evidence": [
        {
            "title": "Abhidharma (wikipedia)",
            "line_idx": 7,
            "text": "These texts developed out of early Buddhist lists or matrices (m\u0101t\u1e5bk\u0101s) of key teachings.",
            "sim": 0.5447394847869873,
            "in_intro": true
        },
        {
            "title": "Abhidharma (wikipedia)",
            "line_idx": 0,
            "text": "The Abhidharma are a collection of Buddhist texts dating from the 3rd century BCE onwards, which contain detailed scholastic presentations of doctrinal material appearing in the canonical Buddhist scriptures and com

# 8 BM25 Selection (Whole Page vs. Intro Page) (Best Setup)

In our document selection process, we first retrieve all pages related to a specific word. Given the potentially large number of pages and the associated computation time, we use the BM25 algorithm to select the top 3 most relevant pages for the claim.

In this section, we evaluate whether using only the intro sections of these pages is sufficient for effective BM25 selection, which also reduces computation time. We will compare the results of selecting from intro sections versus the full pages to determine if the selections are consistent and if the top-ranked pages are the same in both scenarios.


We check on:
- All Evidence Match
- First Evidence Match
- First Summary Evidence in Whole Page Evidences
- First Whole Page Evidence in in Summary Evidences

In [11]:
for dataset_name, config in datasets.items():
    print(f"Evaluating {dataset_name}...")
    dataset = config['dataset']
    lang = config['lang']

    equal_evids = 0
    first_evid_match = 0
    first_intro_match = 0
    first_full_match = 0
    for entry in tqdm(dataset):
        if entry['in_wiki'] == 'No':
            continue

        _, intro_evids = offline_evid_fetcher.fetch_evidences(
            search_word=entry['document_search_word'], only_intro=True)
        _, full_evids = offline_evid_fetcher.fetch_evidences(
            search_word=entry['document_search_word'], only_intro=False)

        intro_evids_indices = rank_docs(entry['connected_claim'],
                                        [" ".join(evidence.get('lines')) for evidence in
                                         intro_evids], k=3)
        selected_intro_pages = [intro_evids[idx].get('title') for idx in intro_evids_indices]
        full_evids_indices = rank_docs(entry['connected_claim'],
                                       [" ".join(evidence.get('lines')) for evidence in full_evids],
                                       k=3)
        selected_full_pages = [full_evids[idx].get('title') for idx in full_evids_indices]

        if set(selected_intro_pages) == set(selected_full_pages):
            equal_evids += 1
        if selected_intro_pages[0] == selected_full_pages[0]:
            first_evid_match += 1
        if selected_intro_pages[0] in selected_full_pages:
            first_intro_match += 1
        if selected_full_pages[0] in selected_intro_pages:
            first_full_match += 1

    print(f"All Evidence Match: {round(100 * equal_evids / len(dataset), 2)}%")
    print(f"First Evidence Match: {round(100 * first_evid_match / len(dataset), 2)}%")
    print(f"First Intro in Full Pages: {round(100 * first_intro_match / len(dataset), 2)}%")
    print(f"First Full Page in Intro: {round(100 * first_full_match / len(dataset), 2)}%")

Evaluating german_dpr-claim_verification...


100%|██████████| 168/168 [05:34<00:00,  1.99s/it]


All Evidence Match: 58.93%
First Evidence Match: 56.55%
First Intro in Full Pages: 78.57%
First Full Page in Intro: 76.19%
Evaluating german_wiktionary-claim_verification-mini...


 12%|█▏        | 23/200 [00:42<06:37,  2.25s/it]

Imam/German/noun: DEBUG: unrecognized sense qualifier: Twelver Shiism at ['Imam']
imam/English/noun: DEBUG: unrecognized sense qualifier: Twelver Shi'ism at ['imam']
Imam/German/noun: DEBUG: unrecognized sense qualifier: Twelver Shiism at ['Imam']
imam/English/noun: DEBUG: unrecognized sense qualifier: Twelver Shi'ism at ['imam']


...

piss/English/noun: DEBUG: gloss may contain unhandled list items: 1999, Tin House #2 (→ISBN, Win McCormack, Rob Spillman, Elissa Schappell), page 170: at ['piss']
piss/English/noun: DEBUG: gloss may contain unhandled list items: 1999, Tin House #2 (→ISBN, Win McCormack, Rob Spillman, Elissa Schappell), page 170: at ['piss']


 58%|█████▊    | 329/563 [02:17<01:15,  3.08it/s]

demosophy/English/noun: DEBUG: unrecognized sense qualifier: folkloristics, sociology, rare at ['demosophy']
demosophy/English/noun: DEBUG: unrecognized sense qualifier: folkloristics, sociology, rare at ['demosophy']


100%|██████████| 563/563 [04:13<00:00,  2.22it/s]

All Evidence Match: 87.21%
First Evidence Match: 84.01%
First Intro in Full Pages: 93.43%
First Full Page in Intro: 93.25%


# 9 GermanDPR: Analysis of Sentence and Document Selection on Accuracy
We assumed that a word definition can be effectively evaluated using up to three sentences. Additionally, we select the top three Wiki articles using the BM25 algorithm before passing them to the evidence selector. 

Here, we test the accuracy across combinations of selected sentences and documents, with selections ranging from one to five.

In [ ]:
output_file_base_name = str(EVALUATION_DIR / "german_dpr-claim_verification/hyperparam/german_dpr-claim_verification_finetuned_soft_threshold_evd{max_evidence}_top{top_k}")

for max_evidence_count in range(1, 6):
    for top_k in range(1, 6):
        print(f"Evaluating with max evidence: {max_evidence_count}, top k sentences: {top_k}...")
        config = datasets.get('german_dpr-claim_verification')
        dataset =config['dataset']
        lang = config['lang']
        pipeline = Pipeline(translator=translator,
                            sent_connector=phi_sentence_connector,
                            claim_splitter=None,
                            evid_fetcher=offline_evid_fetcher,
                            evid_selector=pipeline_models['finetuned_soft_threshold']['evid_selector'],
                            stm_verifier=pipeline_models['finetuned_soft_threshold']['stm_verifier'],
                            lang=lang)
        outputs = evaluate_pipeline(pipeline, dataset, only_intro=True, max_evidence_count=max_evidence_count, top_k=top_k, output_file_name=output_file_base_name.format(max_evidence=max_evidence_count, top_k=top_k))

Evaluating with max evidence: 1, top k sentences: 1...


100%|██████████| 42/42 [00:46<00:00,  1.12s/it]


################################
Not in wikipedia: 29
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.6465    0.9275    0.7619        69
           1     0.8750    0.5000    0.6364        70

    accuracy                         0.7122       139
   macro avg     0.7607    0.7138    0.6991       139
weighted avg     0.7616    0.7122    0.6987       139

################################
Evaluating with max evidence: 1, top k sentences: 2...


100%|██████████| 42/42 [00:48<00:00,  1.16s/it]


################################
Not in wikipedia: 29
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.7647    0.9420    0.8442        69
           1     0.9259    0.7143    0.8065        70

    accuracy                         0.8273       139
   macro avg     0.8453    0.8282    0.8253       139
weighted avg     0.8459    0.8273    0.8252       139

################################
Evaluating with max evidence: 1, top k sentences: 3...


...

################################
Not in wikipedia: 29
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.8356    0.8841    0.8592        69
           1     0.8788    0.8286    0.8529        70

    accuracy                         0.8561       139
   macro avg     0.8572    0.8563    0.8560       139
weighted avg     0.8574    0.8561    0.8560       139

################################
Evaluating with max evidence: 5, top k sentences: 4...


100%|██████████| 42/42 [00:57<00:00,  1.38s/it]


################################
Not in wikipedia: 29
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.8451    0.8696    0.8571        69
           1     0.8676    0.8429    0.8551        70

    accuracy                         0.8561       139
   macro avg     0.8564    0.8562    0.8561       139
weighted avg     0.8564    0.8561    0.8561       139

################################
Evaluating with max evidence: 5, top k sentences: 5...


100%|██████████| 42/42 [00:57<00:00,  1.37s/it]

################################
Not in wikipedia: 29
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.8806    0.8551    0.8676        69
           1     0.8611    0.8857    0.8732        70

    accuracy                         0.8705       139
   macro avg     0.8709    0.8704    0.8704       139
weighted avg     0.8708    0.8705    0.8705       139

################################


# 10 Sentence Ordering / Selection on Testsets
The performance of individual hyperparameters showed minimal variation. Therefore, we evaluate the impact of different sentence selection and ordering methods on our test sets by testing each possible combination. The evidence selection threshold is fixed at 0.5 throughout.

In [ ]:
sentence_selection = ['mmr', 'top']
sentence_order = ['reverse', 'keep', 'top_last']
pipeline = Pipeline(translator=translator,
                    sent_connector=phi_sentence_connector,
                    claim_splitter=None,
                    evid_fetcher=offline_evid_fetcher,
                    evid_selector=pipeline_models['finetuned_soft_threshold']['evid_selector'],
                    stm_verifier=pipeline_models['finetuned_soft_threshold']['stm_verifier'],
                    lang='')

In [ ]:
output_file_base_name = str(EVALUATION_DIR / "{dataset}/pipeline/{dataset}_finetuned_soft_threshold_{selection}_{order}")

for selection in sentence_selection:
    for order in sentence_order:
        print(f"Evaluating with sentence selection: {selection}, sentence order: {order}...")
        pipeline.evid_selector.set_evidence_selection(selection)
        pipeline.stm_verifier.set_premise_sent_order(order)
        for dataset_name, config in datasets.items():
            print(f"Evaluating {dataset_name}...")
            dataset = config['dataset']
            pipeline.lang = config['lang']
            evaluate_pipeline(pipeline, dataset, only_intro=True, output_file_name=output_file_base_name.format(dataset=dataset_name, selection=selection, order=order))

Evaluating with sentence selection: mmr, sentence order: reverse...
Evaluating german_dpr-claim_verification...


100%|██████████| 42/42 [00:55<00:00,  1.33s/it]


################################
Not in wikipedia: 29
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.8289    0.9130    0.8690        69
           1     0.9048    0.8143    0.8571        70

    accuracy                         0.8633       139
   macro avg     0.8669    0.8637    0.8631       139
weighted avg     0.8671    0.8633    0.8630       139

################################
Evaluating german_wiktionary-claim_verification-mini...


 12%|█▏        | 6/50 [00:06<00:47,  1.08s/it]

Imam/German/noun: DEBUG: unrecognized sense qualifier: Twelver Shiism at ['Imam']
imam/English/noun: DEBUG: unrecognized sense qualifier: Twelver Shi'ism at ['imam']


...

piss/English/noun: DEBUG: gloss may contain unhandled list items: 1999, Tin House #2 (→ISBN, Win McCormack, Rob Spillman, Elissa Schappell), page 170: at ['piss']


 58%|█████▊    | 82/141 [01:02<00:37,  1.57it/s]

demosophy/English/noun: DEBUG: unrecognized sense qualifier: folkloristics, sociology, rare at ['demosophy']


100%|██████████| 141/141 [01:50<00:00,  1.28it/s]

################################
Not in wikipedia: 21
Avg claim count: 1.0
              precision    recall  f1-score   support

           0     0.7028    0.5360    0.6082       278
           1     0.6091    0.7614    0.6768       264

    accuracy                         0.6458       542
   macro avg     0.6560    0.6487    0.6425       542
weighted avg     0.6572    0.6458    0.6416       542

################################
